In [ ]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [9]:
import function
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()
actor = function.actor_picker() 
location = function.location_picker()
theme = function.theme_picker()

system_template = "generates a story centered around the chosen Actor, Location, and Theme.Please provide a {responseType} response." 

input = f"Create a story where {actor} is the main character in {location}, and the story follows a {theme} theme."
# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", input)
    ]
)

# prompt Template also implements runnable and can be easily chained.
model = ChatCohere(model="command-r-plus")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})


"In the bustling city of Bengaluru, India, the renowned actor Amitabh Bachchan stars as Dr. Ajay Singh, a brilliant scientist and inventor, in a thrilling sci-fi adventure. \n\nDr. Singh, known for his groundbreaking work in artificial intelligence, has created a revolutionary humanoid robot named RITA, capable of incredible processing power and equipped with advanced AI. As he prepares to unveil his creation to the world, little does he know that a powerful corporation has its eyes on RITA for their nefarious purposes. \n\nOne night, as Dr. Singh works late in his laboratory, located in the heart of Bengaluru's tech hub, he notices strange activity on his computer. Realizing that someone is trying to hack into RITA's system, he springs into action. With his quick thinking, he manages to thwart the intrusion, but it leaves him concerned and curious. \n\nDetermined to get to the bottom of it, Dr. Singh enlists the help of a young and ambitious journalist, Maya (played by a talented upco

In [10]:
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

"Title: The Quantum Conspiracy\n\nIn the bustling city of Bengaluru, India's Silicon Valley, Amitabh Bachchan, a renowned scientist, stumbled upon a cryptic message while working late one night in his lab. The message, encoded in a complex algorithm, hinted at a sinister plot to manipulate the city's quantum infrastructure.\n\nAs Big B, as he's affectionately known, delved deeper into the mystery, he discovered that a group of rogue scientists had infiltrated the city's quantum computing facilities, using the technology to alter the fabric of reality.\n\nThe scientists, led by a charismatic and cunning leader, had created a device capable of manipulating the quantum states of particles, allowing them to bend time and space to their will. Their ultimate goal was to create a temporal loop, trapping Bengaluru in an eternal cycle of chaos and destruction.\n\nWith the help of his trusted colleague, a brilliant young scientist named Shreya, Big B embarked on a perilous mission to stop the ro

In [11]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Add the initial messages to the store
store.add_message(HumanMessage(input))

# Retrieve the stored messages 
messages = await getMessage()

# Model invocation to generate a story based on the prompt
response = model.invoke(messages)  # Call the model with the messages
print("Generated Story:", response.content)  # Display the generated story

# Add the model's response to the store
store.add_message(SystemMessage(response.content))

# allow the user to customize the story

user_input = "surprising twist at the end"
user_input1 = f"change the given story according to user input {user_input}"

# Add the user's customization request to the store
store.add_message(HumanMessage(user_input1))

# Retrieve updated messages after user input
messages = await getMessage()

# Display all stored messages
print("Conversation History:", messages)

# Call the model again with the updated conversation including the user's input
response = model.invoke(messages)
print("Updated Story:", response.content)

# Add the updated model response to the store
store.add_message(SystemMessage(response.content))

Messages retrieved from DB
Generated Story: What an intriguing request! Here's a story where the legendary Amitabh Bachchan takes center stage in Bengaluru, in a sci-fi adventure like no other:

**Title:** "The Quantum Quest of Big B"

**Synopsis:** In the bustling metropolis of Bengaluru, a strange phenomenon has been detected by the city's top scientists. A rift in the space-time continuum has opened, threatening to unravel the very fabric of reality. Enter Amitabh Bachchan, aka "Big B," a renowned actor and local legend, who finds himself at the forefront of this intergalactic crisis.

**Story:**

It's a typical sunny day in Bengaluru when Big B, now in his 70s, receives an unexpected visit from Dr. Nalini, a brilliant physicist from the Indian Institute of Science (IISc). She bursts into his home, frantically explaining that the city is on the brink of disaster. A quantum anomaly has appeared, causing distortions in the space-time continuum. The anomaly, dubbed "The Bengaluru Bubbl